# Golden State Seal

In this Jupyter notebook the various qualifications for what is needed for a student to earn the Golden State Seal are looked at in order to determine if they have earned this after their junior year. Whether or not they have passed the SBAC tests will be looked at as well as how they performed in the courses from freshman to junior year in order to make these determinations.

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# LIST STU SC ID CID LN FN GR IF GR = 12

students = pd.read_excel(r"C:\Users\derek.castleman\Desktop\Students.xlsx")

sbac = pd.read_csv(r"C:\Users\derek.castleman\Desktop\CERS_SBAC_2023-24_6_20_24ALL.csv")

# LIST STU HIS CRS STU.ID STU.NM STU.GR HIS.CN CRS.CO HIS.TE HIS.YR HIS.MK CRS.E1 HIS.ST 

transcripts = pd.read_excel(r"C:\Users\derek.castleman\Desktop\Transcripts Golden.xlsx")

# LIST CRS CO E1 U1 DC CL (BC courses are 24)

bc_courses = pd.read_excel(r"C:\Users\derek.castleman\Desktop\Courses.xlsx")

In [ ]:
students

In [ ]:
transcripts

In [ ]:
# Selects the transcripts of only the 12th graders
transcripts = transcripts[transcripts['Grade'] == 12]
transcripts

In [ ]:
a = input('Two year number of beginning of freshman year = ') #Input freshman year
a = int(a) # Converting the input into an integer

In [ ]:
b = input('Two year number of beginning of senior year = ') #Input senior year
b = int(b) # Converting the input into an integer

In [ ]:
# Selects transcript grades from freshman through junior year
transcripts = transcripts[(transcripts['Year'] >= a) & (transcripts['Year'] < b)]
transcripts

## SBAC

The performance of the students will be looked at for how they did on their assessment during their junior year. If they earned a score above 3 then they will be labelled as meeting this requirement and the rest will be labelled as a no.

In [ ]:
sbac

In [ ]:
# Selects columns of interest
sbac = sbac[['StudentIdentifier', 'AssessmentType', 'Subject', 'ScaleScoreAchievementLevel']]
sbac

In [ ]:
# Selects the summative tests
sbac = sbac[sbac['AssessmentType'] == 'Summative']
sbac

In [ ]:
# The ELA test is selected
sbac_ela = sbac[sbac['Subject'] == 'ELA']
sbac_ela

In [ ]:
# Students who passed are selected and then assigned that value
sbac_ela_passed = sbac_ela[sbac_ela['ScaleScoreAchievementLevel'] >=3]
sbac_ela_passed['SBAC ELA'] = 'Yes'
sbac_ela_passed

In [ ]:
# Students who did not pass are given that value
sbac_ela_failed = sbac_ela[sbac_ela['ScaleScoreAchievementLevel'] <3]
sbac_ela_failed['SBAC ELA'] = 'No'
sbac_ela_failed

In [ ]:
# Both groups of ela are put back together
sbac_ela = pd.concat([sbac_ela_passed, sbac_ela_failed])
sbac_ela

In [ ]:
# Math tests are selected
sbac_math = sbac[sbac['Subject'] == 'Math']
sbac_math

In [ ]:
# Students who passed math are assigned that value
sbac_math_passed = sbac_math[sbac_math['ScaleScoreAchievementLevel'] >=3]
sbac_math_passed['SBAC Math'] = 'Yes'
sbac_math_passed

In [ ]:
# Students who did not pass math are assigned that value
sbac_math_failed = sbac_math[sbac_math['ScaleScoreAchievementLevel'] <3]
sbac_math_failed['SBAC Math'] = 'No'
sbac_math_failed

In [ ]:
# Both math groups are selected
sbac_math = pd.concat([sbac_math_passed, sbac_math_failed])
sbac_math

In [ ]:
# ELA and Math dataframed are merged together
sbac = pd.merge(sbac_ela, sbac_math, how='outer', on='StudentIdentifier')
sbac

In [ ]:
# Narroed down to whether or not they passed the test
sbac = sbac[['StudentIdentifier', 'SBAC ELA', 'SBAC Math']]
sbac

In [ ]:
students

In [ ]:
# The state tests are merged with the student roster
students_sbac = pd.merge(students, sbac, how='left', left_on = 'State Student ID', right_on='StudentIdentifier')
students_sbac

In [ ]:
students_sbac = students_sbac.drop('StudentIdentifier', axis=1)
students_sbac

## Transcipt Cleaning

The transcripts are cleaned up to determine the top two grades that a student earned in any course that they have taken if it is a non-bc course while the top single grade is chosen for any bc course that they have taken.

In [ ]:
transcripts

In [ ]:
# Only uc/csu courses are selected
transcripts = transcripts[transcripts['CSU Req'].notna()]
transcripts

In [ ]:
bc_courses = bc_courses.rename(columns={'Course ID':'Course#'})
bc_courses

In [ ]:
# The bc course flags are merged with the transcripts
merged = pd.merge(transcripts, bc_courses, how='left', on=['Course title', 'CSU Req'])
merged

In [ ]:
# Non-bc courses are selected
minus_bc  = merged[merged['Crs Lvl'] != 24]
minus_bc

In [ ]:
# The top two grades in any given course are selected
grades_sorted_not_bc = minus_bc.sort_values(['Student ID','Course title','Mark']).groupby(['Student ID','Course title']).head(2)
grades_sorted_not_bc

In [ ]:
# The bc courses are selected
bc_course = merged.loc[merged['Crs Lvl'] == 24]
bc_course

In [ ]:
# The top course in any bc course is selected
bc_sorted = bc_course.sort_values(['Student ID','Course title','Mark']).groupby(['Student ID','Course title']).head(1)
bc_sorted

In [ ]:
# The non bc courses and bc courses are put back together
grades = pd.concat([grades_sorted_not_bc, bc_sorted])

In [ ]:
grades

### ELA

The courses that meet the uc/csu requirment for ELA are selected. Then the terms in which a student earned a B+ and above are looked at and then the number of courses that have two terms meeting this requirement are calculated for each student

In [ ]:
# Courses meeting the ELA requirement are selected
ela = grades[grades['CSU Req'] == 'B']
ela

In [ ]:
# The courses with a B+ or above are looked at
ela_passed = ela[(ela['Mark'] == 'B+') | (ela['Mark'] == 'A-') | (ela['Mark'] == 'A') | (ela['Mark'] == 'A+')]
ela_passed

In [ ]:
# Assigned a value of one for passing the term
ela_passed['ELA Terms'] = 1
ela_passed

In [ ]:
# Courses are group and then the total number of terms passed are calculated
ela_passed = ela_passed.groupby(['Student ID', 'Course title'])['ELA Terms'].sum().reset_index()
ela_passed

In [ ]:
# The classes with two terms passed are selected
ela_passed = ela_passed[ela_passed['ELA Terms'] > 1]
ela_passed

In [ ]:
# Given a value of one ELA course being completely passed
ela_passed['ELA'] = 1
ela_passed

In [ ]:
# The total number of ELA courses are passed
ela = ela_passed.groupby(['Student ID'])['ELA'].sum().reset_index()
ela

### Math

A similar process with the math courses is carried out as with the ELA courses to figure out how many Math courses were passed.

In [ ]:
# Classes meeting the math requirement are selected
math = grades[grades['CSU Req'] == 'C']
math

In [ ]:
# Selects math courses that were a B+ and above
math_passed = math[(math['Mark'] == 'B+') | (math['Mark'] == 'A-') | (math['Mark'] == 'A') | (math['Mark'] == 'A+')]
math_passed

In [ ]:
# Given a value of one for passing the term
math_passed['Math Terms'] = 1
math_passed

In [ ]:
# The number of terms for each course are summed together
math_passed = math_passed.groupby(['Student ID', 'Course title'])['Math Terms'].sum().reset_index()
math_passed

In [ ]:
# Classes in which both terms were passed are selected
math_passed = math_passed[math_passed['Math Terms'] > 1]
math_passed

In [ ]:
# Students are given a value of one for each full year passed
math_passed['Math'] = 1
math_passed

In [ ]:
# The total number of courses passed are summed up for each student
math = math_passed.groupby(['Student ID'])['Math'].sum().reset_index()
math

### Science

Science courses are selected and put through the same process as ELA and Math.

In [ ]:
# Science courses are selected
science = grades[grades['CSU Req'] == 'D']
science

In [ ]:
# Non BC science courses are selected
non_bc = science[science['Crs Lvl'] != 24]
non_bc

In [ ]:
# Courses with a B+ or above are chosen
non_bc = non_bc[(non_bc['Mark'] == 'B+') | (non_bc['Mark'] == 'A-') | (non_bc['Mark'] == 'A') | (non_bc['Mark'] == 'A+')]
non_bc

In [ ]:
# A value of one is given to each term that is passed
non_bc['Science Terms'] = 1
non_bc

In [ ]:
# The total number of terms passed for each course
non_bc = non_bc.groupby(['Student ID', 'Course title'])['Science Terms'].sum().reset_index()
non_bc

In [ ]:
# The courses with two terms passed are selected
non_bc = non_bc[non_bc['Science Terms'] > 1]
non_bc

In [ ]:
# A value of one course passed is given
non_bc['Science'] = 1
non_bc

In [ ]:
# The total number of non bc courses are summed up
non_bc = non_bc.groupby(['Student ID'])['Science'].sum().reset_index()
non_bc

In [ ]:
# BC courses are selected
bc = science[science['Crs Lvl'] == 24]
bc

In [ ]:
# The classes with a grade B+ and above are selected
bc = bc[(bc['Mark'] == 'B+') | (bc['Mark'] == 'A-') | (bc['Mark'] == 'A') | (bc['Mark'] == 'A+')]
bc

In [ ]:
# A value of one course passed is given
bc['Science'] = 1
bc

In [ ]:
# The number of BC science courses are added up
bc = bc.groupby(['Student ID'])['Science'].sum().reset_index()
bc

In [ ]:
# All science courses are added together
science = pd.concat([non_bc, bc])
science

In [ ]:
# The total of science courses are added up
science = science.groupby(['Student ID'])['Science'].sum().reset_index()
science

### History

Whether or not a student passed US History will be looked at. Then the number of other history courses that the student has passed will be calculated.

In [ ]:
# All history courses will be selected
history = grades[grades['CSU Req'] == 'A']
history

In [ ]:
# US History is selected
non_bc = history[history['Course#'] == '3010']
non_bc

In [ ]:
# Selects grades that are B and above
non_bc = non_bc[(non_bc['Mark'] == 'B+') | (non_bc['Mark'] == 'A-') | (non_bc['Mark'] == 'A') | (non_bc['Mark'] == 'A+') | (non_bc['Mark'] == 'B')]
non_bc

In [ ]:
# Gives a value of 1 to each term passed
non_bc['US Terms'] = 1
non_bc

In [ ]:
# Sums up the number of terms passed
non_bc = non_bc.groupby(['Student ID', 'Course title'])['US Terms'].sum().reset_index()
non_bc

In [ ]:
# Selecting courses that have passed both semesters
non_bc = non_bc[non_bc['US Terms'] > 1]
non_bc

In [ ]:
# Given one course passed for each one
non_bc['US History'] = 1
non_bc

In [ ]:
# Summing up the number of US History courses passed
non_bc = non_bc.groupby(['Student ID'])['US History'].sum().reset_index()
non_bc

In [ ]:
# Selecting the US History BC course
bc = history[history['Course#'] == '3020']
bc

In [ ]:
# Selecting students who passed the course with B and above
bc = bc[(bc['Mark'] == 'B+') | (bc['Mark'] == 'A-') | (bc['Mark'] == 'A') | (bc['Mark'] == 'A+') | (bc['Mark'] == 'B')]
bc

In [ ]:
# Given one course passed for each student
bc['US History'] = 1
bc

In [ ]:
bc = bc.groupby(['Student ID'])['US History'].sum().reset_index()
bc

In [ ]:
# Puts together BC and non BC US History courses
us_history = pd.concat([non_bc, bc])

In [ ]:
us_history

In [ ]:
us_history.describe()

In [ ]:
history

In [ ]:
# Filtering out US History
history = history[history['Course#'] != '3010']
history

In [ ]:
# Filtering out the BC US History course
history = history[history['Course#'] != '3020']
history

In [ ]:
# Selecting non BC courses
non_bc = history[history['Crs Lvl'] != 24]
non_bc

In [ ]:
# Selecting courses that are passed with B and above
non_bc = non_bc[(non_bc['Mark'] == 'B+') | (non_bc['Mark'] == 'A-') | (non_bc['Mark'] == 'A') | (non_bc['Mark'] == 'A+') | (non_bc['Mark'] == 'B')]
non_bc

In [ ]:
# Give a value of one for a term passed
non_bc['Other Terms'] = 1
non_bc

In [ ]:
# Sums up the terms passed
non_bc = non_bc.groupby(['Student ID', 'Course title'])['Other Terms'].sum().reset_index()
non_bc

In [ ]:
# Selects students who have passed both terms
non_bc = non_bc[non_bc['Other Terms'] > 1]
non_bc

In [ ]:
# Gives a value of one for course passed
non_bc['Other History'] = 1
non_bc

In [ ]:
# Sums up the history classes passed
non_bc = non_bc.groupby(['Student ID'])['Other History'].sum().reset_index()
non_bc

In [ ]:
# Looks at the BC courses passed
bc = history[history['Crs Lvl'] == 24]
bc

In [ ]:
# Selecting students who earned a B or above
bc = bc[(bc['Mark'] == 'B+') | (bc['Mark'] == 'A-') | (bc['Mark'] == 'A') | (bc['Mark'] == 'A+') | (bc['Mark'] == 'B')]
bc

In [ ]:
# Gives credit for one course passed
bc['Other History'] = 1
bc

In [ ]:
# Sums up number of BC courses passed
bc = bc.groupby(['Student ID'])['Other History'].sum().reset_index()
bc

In [ ]:
# Concats other history courses together
other_history = pd.concat([non_bc, bc])
other_history

In [ ]:
# Sums up all the other history courses passed
other_history = other_history.groupby(['Student ID'])['Other History'].sum().reset_index()
other_history

In [ ]:
# Merges the two history columns together
history = pd.merge(us_history, other_history, how='outer', on='Student ID')
history

In [ ]:
# Fills in nan values with a 0
history = history.fillna(0)
history

### Other Courses

The courses that meet other A through G subjects that have not been looked at will go through a similar process to figure out how many other courses the students have passed.

In [ ]:
# The other categories of A through G are selected
other = grades[(grades['CSU Req'] == 'E') | (grades['CSU Req'] == 'F') | (grades['CSU Req'] == 'G')]
other

In [ ]:
# Non BC courses are selected
non_bc = other[other['Crs Lvl'] != 24]
non_bc

In [ ]:
# Courses where students have earned a B or above are looked at
non_bc = non_bc[(non_bc['Mark'] == 'B+') | (non_bc['Mark'] == 'A-') | (non_bc['Mark'] == 'A') | (non_bc['Mark'] == 'A+') | (non_bc['Mark'] == 'B')]
non_bc

In [ ]:
# Give credit for one semester passed
non_bc['Other Terms'] = 1
non_bc

In [ ]:
# Add up terms for each course passed
non_bc = non_bc.groupby(['Student ID', 'Course title'])['Other Terms'].sum().reset_index()
non_bc

In [ ]:
# Selects courses where only 2 semesters were passed
non_bc = non_bc[non_bc['Other Terms'] > 1]
non_bc

In [ ]:
# Gives credit for one course passed
non_bc['Other Classes'] = 1
non_bc

In [ ]:
# Sums up the number of classes passed
non_bc = non_bc.groupby(['Student ID'])['Other Classes'].sum().reset_index()
non_bc

In [ ]:
# Selects the BC courses
bc = other[other['Crs Lvl'] == 24]
bc

In [ ]:
# Selects the courses where B or above was earned
bc = bc[(bc['Mark'] == 'B+') | (bc['Mark'] == 'A-') | (bc['Mark'] == 'A') | (bc['Mark'] == 'A+') | (bc['Mark'] == 'B')]
bc

In [ ]:
# Gives credit for one course passed
bc['Other Classes'] = 1
bc

In [ ]:
# Sums up number of BC courses passed
bc = bc.groupby(['Student ID'])['Other Classes'].sum().reset_index()
bc

In [ ]:
# Puts together non BC and BC courses passed
other_classes = pd.concat([non_bc, bc])
other_classes

In [ ]:
# Sums up all other classes passed
other_classes = other_classes.groupby(['Student ID'])['Other Classes'].sum().reset_index()
other_classes

In [ ]:
students_sbac

In [ ]:
# Merges with ELA courses passed
final = pd.merge(students_sbac, ela, how='outer', on='Student ID')
final

In [ ]:
# Merges Math courses passed
final = pd.merge(final, math, how='outer', on='Student ID')
final

In [ ]:
# Merges Science courses
final = pd.merge(final, science, how='outer', on='Student ID')
final

In [ ]:
# Merges History courses
final = pd.merge(final, history, how='outer', on='Student ID')
final

In [ ]:
# Merge other classes
final = pd.merge(final, other_classes, how='outer', on='Student ID')
final

In [ ]:
# Drops duplicates based on student ID
final = final.drop_duplicates(subset='Student ID')
final

In [ ]:
# Fill in nan values with a 0
final = final.fillna(0)
final

## Golden State Student Calculations

If a student is eligible for the Golden State Seal will be looked at in the following sections. It will be done by looking at the students in different categories starting from how they performed on their state tests and then work down to how the performed in their coursework.

### Passed Both SBACS

Students who passed both state tests will be looked at allowing for their ela and math courses to be added into other classes. Then it will be looked at if they passed one science course and the us history course to see if they qualify

In [ ]:
# Selects students who passed both state tests
both_tests = final[(final['SBAC ELA'] == 'Yes') & (final['SBAC Math'] == 'Yes')]
both_tests

In [ ]:
# Add ELA course, math courses and other history courses into other classes
both_tests['Other Classes'] = both_tests['Other Classes'] + both_tests['ELA'] + both_tests['Math'] + both_tests['Other History']
both_tests

In [ ]:
# Selects students who passed one science course
both_science = both_tests[both_tests['Science'] > 1]
both_science

In [ ]:
# Add other science courses into their science class
both_science['Other Classes'] = both_science['Other Classes'] + both_science['Science'] - 1
both_science

In [ ]:
# Reduces science to just one course
both_science['Science'] = 1

In [ ]:
# Selects students who passed one or less science
both_fail = both_tests[both_tests['Science'] <= 1]
both_fail

In [ ]:
# Puts back together the two science groupings
both_tests = pd.concat([both_science, both_fail])
both_tests

In [ ]:
# Sets ELA and Math to zero since they are not needed
both_tests['ELA'] = 0
both_tests['Math'] = 0
both_tests

In [ ]:
# Students who are eligible are selected based on coursework criteria
both_tests_yes = both_tests[(both_tests['Science'] >= 1) & (both_tests['US History'] >= 1) & (both_tests['Other Classes'] >= 2)]
both_tests_yes

In [ ]:
# Given a value of yes for being eligible
both_tests_yes['Golden State'] = 'Yes'

In [ ]:
both_tests

In [ ]:
# Students who are not eligible based on criteria are selected
both_tests_no = both_tests[(both_tests['Science'] < 1) | (both_tests['US History'] < 1) | (both_tests['Other Classes'] < 2)]
both_tests_no

In [ ]:
# Given a value of No for being eligible
both_tests_no['Golden State'] = 'No'
both_tests_no

In [ ]:
# Puts both groups back together
both_tests = pd.concat([both_tests_yes, both_tests_no])
both_tests

### ELA Alone

For this section students who passed the ELA state test only are looked at and then have math coursework used to determine their eligibility

In [ ]:
# Filters student who passed ELA but not Math
ela_alone = final[(final['SBAC ELA'] == 'Yes') & (final['SBAC Math'] == 'No')]
ela_alone

In [ ]:
# Adds ELA coursework and other history to other classes
ela_alone['Other Classes'] = ela_alone['Other Classes'] + ela_alone['ELA'] + ela_alone['Other History']
ela_alone

In [ ]:
# Finds students who completed more than one science
ela_science = ela_alone[ela_alone['Science'] > 1]
ela_science

In [ ]:
# Adds the extra science to other classes
ela_science['Other Classes'] = ela_science['Other Classes'] + ela_science['Science'] - 1
ela_science

In [ ]:
# Sets their science course to one
ela_science['Science'] = 1
ela_science

In [ ]:
# Finds that students who completed one or less science
ela_fail = ela_alone[ela_alone['Science'] <= 1]
ela_fail

In [ ]:
# Puts everyone back together again
ela_alone = pd.concat([ela_science, ela_fail])
ela_alone

In [ ]:
# Finds students who passed more than one math course
ela_math = ela_alone[ela_alone['Math'] > 1]
ela_math

In [ ]:
# Adds the extra math to other classes
ela_math['Other Classes'] = ela_math['Other Classes'] + ela_math['Math'] - 1
ela_math

In [ ]:
# Sets math courses just to 1
ela_math['Math'] = 1
ela_math

In [ ]:
# Finds the students who passed just one or less math
ela_fail = ela_alone[ela_alone['Math'] <= 1]
ela_fail

In [ ]:
# Puts all the data back together again
ela_alone = pd.concat([ela_math, ela_fail])
ela_alone

In [ ]:
# ELA is set to zero since they do not require these classes
ela_alone['ELA'] = 0
ela_alone

In [ ]:
# Selects students who meet reuirements
ela_alone_yes = ela_alone[(ela_alone['Science'] >= 1) & (ela_alone['US History'] >= 1) & (ela_alone['Other Classes'] >= 2)
                          & (ela_alone['Math'] >= 1)]
ela_alone_yes

In [ ]:
# Gives them a yes value for meeting requirements
ela_alone_yes['Golden State'] = 'Yes'
ela_alone_yes

In [ ]:
# Selects students who do not meet requirements
ela_alone_no = ela_alone[(ela_alone['Science'] < 1) | (ela_alone['US History'] < 1) | (ela_alone['Other Classes'] < 2)
                          | (ela_alone['Math'] < 1)]
ela_alone_no

In [ ]:
# Gives them a no value for the requirements
ela_alone_no['Golden State'] = 'No'
ela_alone_no

In [ ]:
# Puts both groups back together
ela_alone = pd.concat([ela_alone_yes, ela_alone_no])
ela_alone

### Math

The same process is completed above but for students who only passed the math test and not the ela test.

In [ ]:
# Selects students who only passed the math test
math_alone = final[(final['SBAC ELA'] == 'No') & (final['SBAC Math'] == 'Yes')]
math_alone

In [ ]:
# Adds the math courses into the other classes
math_alone['Other Classes'] = math_alone['Other Classes'] + math_alone['Math'] + math_alone['Other History']
math_alone

In [ ]:
# Selects students who passed more than one science course
math_science = math_alone[math_alone['Science'] > 1]
math_science

In [ ]:
# Adds the extra science courses into other classes
math_science['Other Classes'] = math_science['Other Classes'] + math_science['Science'] - 1
math_science

In [ ]:
# Sets the science courses to a value of one
math_science['Science'] = 1
math_science

In [ ]:
# Selects students who had one or less science courses
math_fail = math_alone[math_alone['Science'] <= 1]
math_fail

In [ ]:
# Put both science groups together
math_alone = pd.concat([math_science, math_fail])
math_alone

In [ ]:
# Selects students who had more than one ela course passed
math_ela = math_alone[math_alone['ELA'] > 1]
math_ela

In [ ]:
# Adds the extra ela classes to the other classes
math_ela['Other Classes'] = math_ela['Other Classes'] + math_ela['ELA'] - 1
math_ela

In [ ]:
# Sets the ela classes to one
math_ela['ELA'] = 1
math_ela

In [ ]:
# Selects student who had one or less ela
math_fail = math_alone[math_alone['ELA'] <= 1]
math_fail

In [ ]:
# Put both groups of ela together
math_alone = pd.concat([math_ela, math_fail])
math_alone

In [ ]:
# Sets math to zero courses passed
math_alone['Math'] = 0
math_alone

In [ ]:
# Selects students who meet requirements
math_alone_yes = math_alone[(math_alone['Science'] >= 1) & (math_alone['US History'] >= 1) & (math_alone['Other Classes'] >= 2)
                          & (math_alone['ELA'] >= 1)]
math_alone_yes

In [ ]:
# Gives a yes to meeting the requirements
math_alone_yes['Golden State'] = 'Yes'
math_alone_yes

In [ ]:
# Selects students who do not meet the requirements
math_alone_no = math_alone[(math_alone['Science'] < 1) | (math_alone['US History'] < 1) | (math_alone['Other Classes'] < 2)
                          | (math_alone['ELA'] < 1)]
math_alone_no

In [ ]:
# Gives a no for meeting the requirements
math_alone_no['Golden State'] = 'No'
math_alone_no

In [ ]:
# Puts both groups together
math_alone = pd.concat([math_alone_yes, math_alone_no])
math_alone

### No Test

In this section, students who did not meet any of the state test requirements are looked at. For these students it is just coursework that is used to determine whether or not they meet eligibility.

In [ ]:
# Students who did not pass either test are looked at
no_test = final[(final['SBAC ELA'] == 'No') & (final['SBAC Math'] == 'No')]
no_test

In [ ]:
# Other history are added into the other classes
no_test['Other Classes'] = no_test['Other Classes'] + no_test['Other History']
no_test

In [ ]:
# Students who passed more than one science course are looked at
no_science = no_test[no_test['Science'] > 1]
no_science

In [ ]:
# The extra science courses are added into other classes
no_science['Other Classes'] = no_science['Other Classes'] + no_science['Science'] - 1
no_science

In [ ]:
# Science is set to one
no_science['Science'] = 1
no_science

In [ ]:
# Students who passed one or less science course are selected
no_fail = no_test[no_test['Science'] <= 1]
no_fail

In [ ]:
# Both science groups are put together
no_test = pd.concat([no_science, no_fail])
no_test

In [ ]:
# Selects students who passed more than one ela
no_ela = no_test[no_test['ELA'] > 1]
no_ela

In [ ]:
# Adds extra ela classes into other classes
no_ela['Other Classes'] = no_ela['Other Classes'] + no_ela['ELA'] - 1
no_ela

In [ ]:
# Sets ela to one
no_ela['ELA'] = 1
no_ela

In [ ]:
# Selects students who passed one or less ela
no_fail = no_test[no_test['ELA'] <= 1]
no_fail

In [ ]:
# Puts both groups together
no_test = pd.concat([no_ela, no_fail])
no_test

In [ ]:
# Selects students who passed more than one math
no_math = no_test[no_test['Math'] > 1]
no_math

In [ ]:
# Adds extra math classes to other classes
no_math['Other Classes'] = no_math['Other Classes'] + no_math['Math'] - 1
no_math

In [ ]:
# Sets math to one
no_math['Math'] = 1
no_math

In [ ]:
# Selects one or under math
no_fail = no_test[no_test['Math'] <= 1]
no_fail

In [ ]:
# Puts both math groups back together again
no_test = pd.concat([no_math, no_fail])
no_test

In [ ]:
# Selects students who meet requirements
no_alone_yes = no_test[(no_test['Science'] >= 1) & (no_test['US History'] >= 1) & (no_test['Other Classes'] >= 2)
                          & (no_test['ELA'] >= 1) & (no_test['Math'] >= 1)]
no_alone_yes

In [ ]:
# Gives a yes for students who meet requirements
no_alone_yes['Golden State'] = 'Yes'
no_alone_yes

In [ ]:
# Selects students who do not meet requirements
no_alone_no = no_test[(no_test['Science'] < 1) | (no_test['US History'] < 1) | (no_test['Other Classes'] < 2)
                          | (no_test['Math'] < 1) | (no_test['ELA'] < 1)]
no_alone_no

In [ ]:
# Gives a no to meeting requirements
no_alone_no['Golden State'] = 'No'
no_alone_no

In [ ]:
# Puts both groups back together
no_alone = pd.concat([no_alone_yes, no_alone_no])
no_alone

### Final

Puts together all of the different groups to produce one single dataframe.

In [ ]:
final = pd.concat([both_tests, ela_alone, math_alone, no_alone])
final

In [ ]:
final = final.drop_duplicates(subset='Student ID', keep='first')
final

In [ ]:
# Gets rid of other history since it is useless at this time
final = final.drop(['Other History'], axis=1)
final

In [ ]:
import base64
from IPython.display import HTML

def create_download_link( df, title = "Golden State", filename = "Golden State"):
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(final)

In [ ]:
#### 